# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115812e+02     1.567620e+00
 * time: 0.8317649364471436
     1     1.131471e+01     1.002168e+00
 * time: 2.2271101474761963
     2    -1.221523e+01     1.041643e+00
 * time: 2.983083963394165
     3    -3.418227e+01     8.421596e-01
 * time: 4.042772054672241
     4    -4.785940e+01     7.182707e-01
 * time: 5.055032968521118
     5    -5.689129e+01     2.557973e-01
 * time: 6.085591077804565
     6    -5.961765e+01     2.253163e-01
 * time: 6.7738869190216064
     7    -6.079319e+01     8.191246e-02
 * time: 7.451801061630249
     8    -6.133884e+01     4.587786e-02
 * time: 8.144854068756104
     9    -6.166177e+01     3.424708e-02
 * time: 8.831918001174927
    10    -6.183103e+01     3.470563e-02
 * time: 9.533293008804321
    11    -6.198891e+01     2.730538e-02
 * time: 10.207878112792969
    12    -6.207985e+01     1.952545e-02
 * time: 10.889554023742676
    13    -6.213182e+01     1.494721e-02
 * time: 11.585948944091797
 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671064
    AtomicLocal         -18.8557660
    AtomicNonlocal      14.8522634
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336820

    total               -62.261666453138
